# Importing Libraries

In [3]:
import sys
import pyrootutils

root = pyrootutils.setup_root(sys.path[0], pythonpath=True, cwd=True)

# import timm
import torch
import shutil
import numpy as np
import torchvision
# import seaborn as sns
import torch.nn as nn
# import albumentations as A
import torch.optim as optim
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transforms

from PIL import Image
from pathlib import Path
from omegaconf import OmegaConf
from torchvision import datasets
from torchmetrics import Accuracy
from hydra import compose, initialize
from torch import nn, optim, utils, Tensor
# from albumentations.pytorch import ToTensorV2
from torch.utils.data import random_split, DataLoader, TensorDataset


# shutil.copy("configs/config.yaml", "notebooks/config.yaml")
# with initialize(version_base=None, config_path=""):
#     config = compose(config_name="config.yaml")


/home/shivam13juna/Documents/virtual_envs/capstone/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
from src.datamodules import intel_datamodule

ModuleNotFoundError: No module named 'src'

# Loading Data

In [2]:
root_dir = Path("data/")
files = list(root_dir.glob("*/*/*/*"))
label_file = list(map(lambda x: x.parent.stem, files))

In [38]:
train_files = list((root_dir / "seg_train" / "seg_train").glob("*/*"))
train_label = list(map(lambda x: x.parent.stem, train_files))

# Load an image from train_files using pytorch and apply transforms
img = Image.open(train_files[0])
img = transforms.ToTensor()(img)


In [59]:
# ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
label_dict = {'buildings':0, 'forest':1, 'glacier':2, 'mountain':3, 'sea':4, 'street':5}

mean_intel = [0.4302, 0.4575, 0.4539]
std_intel = [0.2600, 0.2585, 0.2904]

In [60]:
# Path: notebooks/data_stats.ipynb

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, data_files, label_dict, transform=None):

        self.root_dir = Path(root_dir)
        self.data_files = data_files
        self.transform = transform       
        self.label_dict = label_dict 

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, index):

        # Load the image and label at the given index
        image = np.array(Image.open(self.data_files[index]))
        # convert to numpy array

        label = self.label_dict[self.data_files[index].parent.stem]

        if self.transform:
            image = self.transform(image)

        return image, label


In [61]:
train_files = list((root_dir / "seg_train" / "seg_train").glob("*/*"))
# train_labels = list(map(lambda x: x.parent.stem, train_files))

test_files = list((root_dir / "seg_test" / "seg_test").glob("*/*"))
# test_labels = list(map(lambda x: x.parent.stem, test_files))

im_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((32, 32)), transforms.Normalize(mean_intel, std_intel)])

train_dataset = ImageDataset(root_dir, train_files,  label_dict=label_dict, transform=im_transform)


train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [62]:
for x, y in train_dataloader:
    print(x.shape, y.shape)
    break

torch.Size([32, 3, 32, 32]) torch.Size([32])
